Notebook to perform meta learning from machine learning. Here, we take the 5 best models from the 5-folds, load their predictions on the validation set, use them as inputs (size (5,1)) and predict the original outputs. We test on the total training set that is used for multi-task ML methods

In [ ]:
import os

if "CUDA_VISIBLE_DEVICES" not in os.environ:
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(
            gpu, True
        )  # Limiting the memory growth
    logical_gpus = tf.config.experimental.list_logical_devices("GPU")
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

import numpy as np

SEED = 42
import random


def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)


def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    # tf.config.threading.set_inter_op_parallelism_threads(1)
    # tf.config.threading.set_intra_op_parallelism_threads(1)


# Call the above function with seed value
set_global_determinism(seed=SEED)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import optuna
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold

Loading and splitting

In [2]:
data=pd.read_json('/home/subhash/test/batterie_informatik/ne/Organic_battery_dataset.json')

In [3]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold

class MyMinMaxScaler(MinMaxScaler):
    def inverse_transform(self, X):
        X -= self.min_
        X /= self.scale_
        return X

def log10(x):
    return np.log10(x + 1)


def log10_inv(x):
    return (10 ** x) - 1

def get_property_scalers(props):
    log10_list = ["sc"]
    # log first then minmax
    scalers = {}
    for propertys in props:
        mm1 = MyMinMaxScaler()
        scalers[propertys] = Pipeline([("minmax", mm1)])
    return scalers

def get_datasets(df_init):
    df = df_init.copy()

    df["scaled_value"] = df["value"]

    props = df['Property'].unique().tolist()

    # Create final dataset for ensemble
    ps_final = get_property_scalers(props)

    for prop in props:
        # train
        cond = df[df['Property'] == prop].index
        val = df.loc[cond, ["scaled_value"]]
        df.loc[cond, ["scaled_value"]] = ps_final[prop].fit_transform(val)

    training_df,val_df=train_test_split(df,test_size=0.2,random_state=0,stratify=df['scala'])
 
    k_fold_datasets = []
    skf = StratifiedKFold(n_splits=5,shuffle=False)
    for train_index, val_index in skf.split(training_df, training_df['scala']):

        k_train = training_df.iloc[train_index].copy()
        k_val = training_df.iloc[val_index].copy()

        k_fold_datasets.append({"train": k_train, "val": k_val, "scaler":ps_final})

    return training_df,k_fold_datasets,val_df,ps_final

In [4]:
train, total, val, scaler = get_datasets(data)

In [ ]:
import optuna 
from optuna import trial
from pathlib import Path
path=Path('/home/subhash/test/batterie_informatik/ne/Multi_task_models')
tr_inputs=tf.convert_to_tensor(list(val.cfp))
tr_outputs=tf.convert_to_tensor(list(val.scaled_value))
te_inputs=tf.convert_to_tensor(list(train.cfp))
te_outputs=tf.convert_to_tensor(list(train.scaled_value))
tr_i,te_i=[],[]
for i in range(5):
    mod=tf.keras.models.load_model(f'{path}/{i}.keras')
    tr_i.append(mod.predict(tr_inputs))
    te_i.append(mod.predict(te_inputs))

In [7]:
a=np.reshape(tr_i,newshape=(5,len(tr_i[0])))

In [8]:
b=np.reshape(te_i,newshape=(5,len(te_i[0])))

In [9]:
meta_train=a.T


In [10]:
meta_test=b.T

In [11]:
#Train with val and val with train
#Take all the models
#do predictions
#store them in dataframe
#Make them as inputs and then train a model in optuna.


import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.metrics import R2Score
import itertools
from optuna.integration import TFKerasPruningCallback
import numpy as np 

seed = 123
batch_size = 200

np.random.seed(seed)
tf.random.set_seed(seed)

def create_model(trial):
    
    n_layers = trial.suggest_int("n_layers", 2, 6)
    model = tf.keras.Sequential()
    # model.add(tf.keras.layers.Dense(64))
    for num in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(num), 64, 512, step=64)
        model.add(tf.keras.layers.Dense(
                num_hidden,
            )) # Adding dense and dropout alternatively
        
        activation_name = trial.suggest_categorical(
            f"actvtn_l{num}", ['relu','LeakyReLU','selu','gelu', 'prelu']
        )
        if activation_name == "prelu":
            activation_layer=tf.keras.layers.PReLU()      
        elif activation_name == "LeakyReLU":
            activation_layer=tf.keras.layers.LeakyReLU()
        else:
            activation_layer=tf.keras.layers.Activation(activation_name)

        model.add(activation_layer)

        n_drop = trial.suggest_float("n_drop_l{}".format(num), 0, 0.7)
        model.add(tf.keras.layers.Dropout(n_drop))
    
    #model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1))
    return model

def objective(trial):
    train_dataset=tf.data.Dataset.from_tensor_slices((meta_train,tr_outputs))
    test_dataset=tf.data.Dataset.from_tensor_slices((meta_test,te_outputs))
    learning_rate = trial.suggest_float("learning_rate", 5e-4, 5e-3, log=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    # dataset=dataset
    metric = R2Score()
    model = create_model(trial)
    #rmse=mean_squared_error(squared=False)
    model.compile(optimizer=optimizer, loss="mse", metrics=[metric])
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.85, monitor="val_loss", verbose=True
        ),
        tf.keras.callbacks.EarlyStopping(patience=40, monitor="val_loss", verbose=True)]
       
    train_dataset = (
        train_dataset.cache().batch(50).prefetch(tf.data.experimental.AUTOTUNE)
            )  
    test_dataset = (
            test_dataset.cache().batch(50).prefetch(tf.data.experimental.AUTOTUNE)
            )

    fit = model.fit(
        train_dataset,epochs=500, validation_data=(test_dataset),callbacks=callbacks,batch_size=50)
    from pathlib import Path
    pt = Path(f'meta_learner/{trial.number}.keras')
    pt.parent.mkdir(parents=True, exist_ok=True)
    model.save(pt)
    return fit.history['val_loss'][-1]


In [ ]:
from functools import partial
import optuna
_dir = locals().get("_dir", None)
if not _dir:
    _dir = 'meta_learner'

study = optuna.create_study(study_name='meta_learner',
direction="minimize",storage=f"sqlite:///{_dir}.db",load_if_exists=True)

study.optimize(objective,n_trials=1000)
